## Finding Bears at SFU ([pbclient](https://github.com/PyBossa/pybossa-client) version)

#### Step 1: Initialization

In [ ]:
import pbclient
from string import Template

pbclient.set('endpoint', 'http://ec2-54-200-84-187.us-west-2.compute.amazonaws.com:5000')
# You can find your api key by clicking on your name, and then in the My Settings option.
pbclient.set('api_key', 'Replace with your own API key') 


# !! The name and short_name of the project must be unique! 
# Otherwise you will get an error (IntegrityError) when creating the project.
name = "Finding Bears at SFU (pbclient)"
short_name = "imglabel-pbclient"
description = "image labeling"
pbclient.create_project(name, short_name, description)

project = pbclient.find_project(short_name=short_name)[0]
template = open('presenter.html').read()
project.info['task_presenter'] = Template(template).safe_substitute( \
                                    short_name = short_name, question = "Do you see a bear in the photo?")
project = pbclient.update_project(project)

print project

#### Step 2: Publishing Tasks

In [ ]:
pics = ['https://github.com/sfu-db/cmpt884-fall16/raw/master/Assignments/A1/3.jpg',
        'https://github.com/sfu-db/cmpt884-fall16/raw/master/Assignments/A1/4.jpg',
        'https://github.com/sfu-db/cmpt884-fall16/raw/master/Assignments/A1/5.jpg']

tid_to_answers = {}
tid_to_pics = {}
for pic in pics:
    task = pbclient.create_task(project.id, {'url_b': pic})
    tid_to_answers[task.id] = None
    tid_to_pics[task.id] = pic

print "Publishing %d tasks" %(len(tid_to_answers))

#### Step 3: Collecting Results

In [ ]:
import time

# Checking whether the tasks has been finished every 3 secs
while None in tid_to_answers.values():
    results = pbclient.get_taskruns(project.id, limit = 100, last_id = 0)
    for result in results:
        task_id = result.data['task_id']
        answer = result.data['info']
        if task_id in tid_to_answers.keys():
            tid_to_answers[task_id] = answer
    time.sleep(3)
    
for tid, pic in tid_to_pics.items():
    print pic, tid_to_answers[tid]